# ELEC/COMP 576 – Task 2 Notebook
**MATLAB ↔ NumPy equivalents**


In [1]:
import numpy as np
from numpy.random import default_rng
from scipy import linalg as la
from scipy import signal
from scipy.sparse import csc_matrix, diags
from scipy.sparse.linalg import eigs, cg

# ============================================================
# ELEC/COMP 576 – Task 2: MATLAB ↔ NumPy equivalents (ALL ITEMS)
# Polished formatting: clear headers, aligned labels, compact shapes.
# Paste outputs directly into your report.
# ============================================================



In [2]:
# ---- Pretty printing helpers -------------------------------------------------
np.set_printoptions(
    precision=4, suppress=True, linewidth=120, edgeitems=2, threshold=60
)

LINE = 80


def hdr(title: str) -> None:
    line = "=" * LINE
    print(f"\n{line}\n{title}\n{line}")


def sub(title: str) -> None:
    print(f"\n{title}\n" + "-" * len(title))


def kv(label: str, value, show_shape: bool = False) -> None:
    shape_str = ""
    if show_shape and hasattr(value, "shape"):
        shape_str = f"  (shape={tuple(value.shape)})"
    print(f"{label:<38} {value}{shape_str}")


def arr(label: str, value, show_shape: bool = True) -> None:
    shape_str = (
        f" (shape={tuple(value.shape)})"
        if show_shape and hasattr(value, "shape")
        else ""
    )
    print(f"{label}:{shape_str}\n{value}")




In [3]:
# ---- Baseline arrays ---------------------------------------------------------
hdr("Baseline arrays")
A = np.arange(1, 25 * 12 + 1).reshape(25, 12).astype(float)  # 25x12
v = np.linspace(0, 1, A.shape[1])  # length-12 mask for columns
v_col = v[:, None]  # (12,1)
kv("Array A: ", A)
kv("Array v: ", v)
kv("A.shape", A.shape)
kv("v.shape / v_col.shape", f"{v.shape} / {v_col.shape}")




Baseline arrays
Array A:                               [[  1.   2. ...  11.  12.]
 [ 13.  14. ...  23.  24.]
 ...
 [277. 278. ... 287. 288.]
 [289. 290. ... 299. 300.]]
Array v:                               [0.     0.0909 0.1818 0.2727 0.3636 0.4545 0.5455 0.6364 0.7273 0.8182 0.9091 1.    ]
A.shape                                (25, 12)
v.shape / v_col.shape                  (12,) / (12, 1)


In [4]:
# ---- Dimensions & sizes ------------------------------------------------------
hdr("Dimensions & sizes")
kv("ndims(A) = np.ndim(A) = A.ndim", f"{np.ndim(A)} = {A.ndim}")
kv("numel(A) = np.size(A) = A.size", f"{np.size(A)} = {A.size}")
kv("size(A) = np.shape(A) = A.shape", f"{np.shape(A)} = {A.shape}")
kv("size(A,1) → A.shape[0]", A.shape[0])
kv("size(A,2) → A.shape[1]", A.shape[1])




Dimensions & sizes
ndims(A) = np.ndim(A) = A.ndim         2 = 2
numel(A) = np.size(A) = A.size         300 = 300
size(A) = np.shape(A) = A.shape        (25, 12) = (25, 12)
size(A,1) → A.shape[0]                 25
size(A,2) → A.shape[1]                 12


In [5]:
# ---- Array construction & blocks --------------------------------------------
hdr("Array construction & blocks")
arr("[1 2 3; 4 5 6]", np.array([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]]))
a = np.array([[1, 0], [0, 1]], float)
b = np.array([[2, 2], [2, 2]], float)
c = 3 * np.ones((2, 2))
d = np.eye(2) * 4
blk = np.block([[a, b], [c, d]])
kv("np.block([[a,b],[c,d]])", "constructed", show_shape=False)
kv("blk.shape", blk.shape)




Array construction & blocks
[1 2 3; 4 5 6]: (shape=(2, 3))
[[1. 2. 3.]
 [4. 5. 6.]]
np.block([[a,b],[c,d]])                constructed
blk.shape                              (4, 4)


In [6]:
# ---- Indexing & slicing ------------------------------------------------------
hdr("Indexing & slicing")
vec = np.arange(10)
kv("a(end) → vec[-1]", vec[-1])
kv("a(2,5) → A[1,4]", A[1, 4])
kv("a(2,:) → A[1,:]", A[1, :])
kv("a(1:5,:) → A[:5,:]", A[:5, :])
kv("a(end-4:end,:) → A[-5:,:]", A[-5:, :])
kv("a(1:3,5:9) → A[0:3,4:9]", A[0:3, 4:9])
kv("a([2,4,5],[1,3]) via ix_", A[np.ix_([1, 3, 4], [0, 2])])
kv("a(3:2:21,:) → A[2:21:2,:]", A[2:21:2, :])
kv("a(1:2:end,:) → A[::2,:]", A[::2, :])
kv("flipud(a) → A[::-1,:]", A[::-1, :])
kv(
    "a([1:end 1],:) → A[np.r_[0:len(A),0],:]",
    A[np.r_[0 : len(A), 0], :]
    
)




Indexing & slicing
a(end) → vec[-1]                       9
a(2,5) → A[1,4]                        17.0
a(2,:) → A[1,:]                        [13. 14. 15. 16. 17. 18. 19. 20. 21. 22. 23. 24.]
a(1:5,:) → A[:5,:]                     [[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.]
 [13. 14. 15. 16. 17. 18. 19. 20. 21. 22. 23. 24.]
 [25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35. 36.]
 [37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48.]
 [49. 50. 51. 52. 53. 54. 55. 56. 57. 58. 59. 60.]]
a(end-4:end,:) → A[-5:,:]              [[241. 242. 243. 244. 245. 246. 247. 248. 249. 250. 251. 252.]
 [253. 254. 255. 256. 257. 258. 259. 260. 261. 262. 263. 264.]
 [265. 266. 267. 268. 269. 270. 271. 272. 273. 274. 275. 276.]
 [277. 278. 279. 280. 281. 282. 283. 284. 285. 286. 287. 288.]
 [289. 290. 291. 292. 293. 294. 295. 296. 297. 298. 299. 300.]]
a(1:3,5:9) → A[0:3,4:9]                [[ 5.  6.  7.  8.  9.]
 [17. 18. 19. 20. 21.]
 [29. 30. 31. 32. 33.]]
a([2,4,5],[1,3]) via ix_               [[13. 15

In [7]:
# ---- Transpose / Conjugate transpose ----------------------------------------
hdr("Transpose / Conjugate transpose")
C = A[:3, :3] + 1j * A[:3, :3]
kv("A.' → A.T shape", A.T.shape)
kv("A'  → A.conj().T equals C.T.conj()?", np.allclose(C.conj().T, C.T.conj()))




Transpose / Conjugate transpose
A.' → A.T shape                        (12, 25)
A'  → A.conj().T equals C.T.conj()?    True


In [8]:
# ---- Arithmetic: matrix vs elementwise; divide; power -----------------------
hdr("Arithmetic (matrix vs elementwise)")
X = np.arange(1, 7).reshape(2, 3).astype(float)
Y = np.arange(1, 7).reshape(3, 2).astype(float)
kv("Matrix multiply a*b → X @ Y shape", (X @ Y).shape)
arr("Element-wise multiply a.*b → X*X", X * X)
arr("Element-wise divide a./b → X/X", X / X)
kv("Element-wise power a.^3 → (X**3)[0,0]", (X**3)[0, 0])




Arithmetic (matrix vs elementwise)
Matrix multiply a*b → X @ Y shape      (2, 2)
Element-wise multiply a.*b → X*X: (shape=(2, 3))
[[ 1.  4.  9.]
 [16. 25. 36.]]
Element-wise divide a./b → X/X: (shape=(2, 3))
[[1. 1. 1.]
 [1. 1. 1.]]
Element-wise power a.^3 → (X**3)[0,0]  1.0


In [9]:
# ---- Logical / find / masking ----------------------------------------------
hdr("Logical / find / masking")
mask = A > 0.5
idxs = np.nonzero(mask)
kv("(A>0.5) dtype & count", f"{mask.dtype}, {len(idxs[0])}")
kv("find(A>0.5) → first 5 (r,c)", list(zip(idxs[0][:5], idxs[1][:5])))
kv("a(:,find(v>0.5)) → A[:, v>0.5] shape", A[:, v > 0.5].shape)
kv("a(:,find(v>0.5)) with v as (N,1)", A[:, v_col.ravel() > 0.5].shape)
A_copy = A.copy()
A_copy[A_copy < 0.5] = 0
kv("a(a<0.5)=0 → min(A_copy)", A_copy.min())
kv("a.*(a>0.5) sum", (A * (A > 0.5)).sum())




Logical / find / masking
(A>0.5) dtype & count                  bool, 300
find(A>0.5) → first 5 (r,c)            [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4)]
a(:,find(v>0.5)) → A[:, v>0.5] shape   (25, 6)
a(:,find(v>0.5)) with v as (N,1)       (25, 6)
a(a<0.5)=0 → min(A_copy)               1.0
a.*(a>0.5) sum                         45150.0


In [10]:
# ---- Assignment / copies / flatten ------------------------------------------
hdr("Assignment / copies / flatten")
B = A.copy()
B[:] = 3
kv("a(:)=3 → unique(B)", np.unique(B))
Yref = A
Ycpy = A.copy()
kv("y=x (reference) shares memory?", Yref is A)
kv("y=x.copy() shares memory?", Ycpy is A)
row2_view = A[1, :]
row2_copy = A[1, :].copy()
kv(
    "y=x(2,:) equal? & is view?",
    f"{np.allclose(row2_view, row2_copy)}, {row2_view.base is A}",
)
flat = A.flatten()
flat_F = A.flatten("F")
kv("y=x(:) → flatten shapes", f"{flat.shape} & {flat_F.shape}")




Assignment / copies / flatten
a(:)=3 → unique(B)                     [3.]
y=x (reference) shares memory?         True
y=x.copy() shares memory?              False
y=x(2,:) equal? & is view?             True, True
y=x(:) → flatten shapes                (300,) & (300,)


In [11]:
# ---- Ranges / vectors --------------------------------------------------------
hdr("Ranges / vectors")
arr("1:10 → np.arange(1.,11.)", np.arange(1.0, 11.0))
arr("0:9  → np.arange(10.)", np.arange(10.0))
kv("[1:10]' shape", np.arange(1.0, 11.0)[:, None].shape)




Ranges / vectors
1:10 → np.arange(1.,11.): (shape=(10,))
[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
0:9  → np.arange(10.): (shape=(10,))
[0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
[1:10]' shape                          (10, 1)


In [12]:
# ---- Zeros / Ones / Eye / Diag ----------------------------------------------
hdr("Zeros / Ones / Eye / Diag")
kv("zeros(3,4) shape", np.zeros((3, 4)).shape)
kv("zeros(3,4,5) shape", np.zeros((3, 4, 5)).shape)
kv("ones(3,4) shape", np.ones((3, 4)).shape)
arr("eye(3)", np.eye(3))
kv("diag(A[:3,:3])", np.diag(A[:3, :3]))
dv = np.array([9, 8, 7])
arr("diag(v,0)", np.diag(dv, 0))




Zeros / Ones / Eye / Diag
zeros(3,4) shape                       (3, 4)
zeros(3,4,5) shape                     (3, 4, 5)
ones(3,4) shape                        (3, 4)
eye(3): (shape=(3, 3))
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
diag(A[:3,:3])                         [ 1. 14. 27.]
diag(v,0): (shape=(3, 3))
[[9 0 0]
 [0 8 0]
 [0 0 7]]


In [13]:
# ---- Random (Generator + legacy rand) ---------------------------------------
hdr("Random (Generator + legacy)")
rng = default_rng(42)
arr("default_rng(42).random((3,4))", rng.random((3, 4)))
arr("np.random.rand(3,4) (legacy)", np.random.rand(3, 4))




Random (Generator + legacy)
default_rng(42).random((3,4)): (shape=(3, 4))
[[0.774  0.4389 0.8586 0.6974]
 [0.0942 0.9756 0.7611 0.7861]
 [0.1281 0.4504 0.3708 0.9268]]
np.random.rand(3,4) (legacy): (shape=(3, 4))
[[0.5596 0.3097 0.9587 0.5298]
 [0.9351 0.909  0.3194 0.1792]
 [0.2791 0.0618 0.6154 0.1957]]


In [14]:
# ---- Grids: linspace / meshgrid / mgrid / ogrid / ix_ -----------------------
hdr("Grids: linspace / meshgrid / mgrid / ogrid / ix_")
arr("linspace(1,3,4)", np.linspace(1, 3, 4))
x_m, y_m = np.meshgrid(np.r_[0:9], np.r_[0:6])
kv("meshgrid shapes", f"{x_m.shape} & {y_m.shape}")
x_M, y_M = np.mgrid[0:9, 0:6]
kv("mgrid shapes", f"{x_M.shape} & {y_M.shape}")
x_O, y_O = np.ogrid[0:9, 0:6]
kv("ogrid shapes", f"{x_O.shape} & {y_O.shape}")
Xix, Yix = np.ix_(np.r_[0:9], np.r_[0:6])
kv("ix_ shapes", f"{Xix.shape} & {Yix.shape}")
arr(
    "f(X,Y)=X+Y via ix_ sample (corners)",
    np.array([Xix[0, 0] + Yix[0, 0], Xix[-1, -1] + Yix[-1, -1]]),
)




Grids: linspace / meshgrid / mgrid / ogrid / ix_
linspace(1,3,4): (shape=(4,))
[1.     1.6667 2.3333 3.    ]
meshgrid shapes                        (6, 9) & (6, 9)
mgrid shapes                           (9, 6) & (9, 6)
ogrid shapes                           (9, 1) & (1, 6)
ix_ shapes                             (9, 1) & (1, 6)
f(X,Y)=X+Y via ix_ sample (corners): (shape=(2,))
[ 0 13]


In [15]:
# ---- Tile & Concatenate ------------------------------------------------------
hdr("Tile & Concatenate")
A2x3 = np.arange(1, 7).reshape(2, 3)
kv("repmat(A2x3,2,3) → np.tile shape", np.tile(A2x3, (2, 3)).shape)
kv(
    "[a b] → hstack & column_stack",
    f"{np.hstack((A2x3, A2x3)).shape} & {np.column_stack((A2x3, A2x3[:,0])).shape}",
)
kv(
    "[a; b] → vstack & r_",
    f"{np.vstack((A2x3, A2x3)).shape} & {np.r_[A2x3, A2x3].shape}",
)




Tile & Concatenate
repmat(A2x3,2,3) → np.tile shape       (4, 9)
[a b] → hstack & column_stack          (2, 6) & (2, 4)
[a; b] → vstack & r_                   (4, 3) & (4, 3)


In [16]:
# ---- Max / norms / logicops --------------------------------------------------
hdr("Max / norms / logicops")
kv("max(max(A)) → A.max()", A.max())
kv("max(A) per-column → A.max(0)[:5]", A.max(0)[:5])
kv("max(A,[],2) per-row → A.max(1)[:5]", A.max(1)[:5])
arr("max(A,B) → np.maximum sample", np.maximum(A[:2, :3], (A[:2, :3] - 5)))
vec2 = np.array([3.0, 4.0])
kv("norm(v) → np.linalg.norm", np.linalg.norm(vec2))
boolA = A[:3, :3] % 2 == 0
boolB = A[:3, :3] % 3 == 0
arr("logical_and(a,b)", np.logical_and(boolA, boolB))
intA = np.array([[1, 2], [3, 4]], dtype=int)
intB = np.array([[4, 1], [2, 3]], dtype=int)
arr("bitand(a,b) → a & b", (intA & intB))
arr("bitor(a,b) → a | b", (intA | intB))




Max / norms / logicops
max(max(A)) → A.max()                  300.0
max(A) per-column → A.max(0)[:5]       [289. 290. 291. 292. 293.]
max(A,[],2) per-row → A.max(1)[:5]     [12. 24. 36. 48. 60.]
max(A,B) → np.maximum sample: (shape=(2, 3))
[[ 1.  2.  3.]
 [13. 14. 15.]]
norm(v) → np.linalg.norm               5.0
logical_and(a,b): (shape=(3, 3))
[[False False False]
 [False False False]
 [False False False]]
bitand(a,b) → a & b: (shape=(2, 2))
[[0 0]
 [2 0]]
bitor(a,b) → a | b: (shape=(2, 2))
[[5 3]
 [3 7]]


In [17]:
# ---- inv / pinv / rank / solves (\ and /) -----------------------------------
hdr("inv / pinv / rank / solves (\ and /)")
M = np.array([[3.0, 1.0, 2.0], [2.0, 6.0, 4.0], [0.0, 1.0, 5.0]])
bb = np.array([1.0, 2.0, 3.0])
arr("inv(M)", la.inv(M))
arr("pinv(M)", la.pinv(M))
kv("rank(M)", np.linalg.matrix_rank(M))
arr("x = M\\bb (square) → solve", la.solve(M, bb))
Z = np.vstack([np.ones(10), np.arange(10)]).T  # 10x2 design
y = 3 + 2 * np.arange(10) + 0.01 * np.random.randn(10)
coef, *_ = la.lstsq(Z, y)
arr("x = Z\\y (lstsq) → [intercept, slope]", coef)
A6 = A[:6, :6]
B2 = np.arange(1, 13).reshape(2, 6).astype(float)
X_right = np.linalg.lstsq(A6.T, B2.T, rcond=None)[0].T
kv("b/a → Solve A.T x.T = b.T, X shape", X_right.shape)




inv / pinv / rank / solves (\ and /)
inv(M): (shape=(3, 3))
[[ 0.3611 -0.0417 -0.1111]
 [-0.1389  0.2083 -0.1111]
 [ 0.0278 -0.0417  0.2222]]
pinv(M): (shape=(3, 3))
[[ 0.3611 -0.0417 -0.1111]
 [-0.1389  0.2083 -0.1111]
 [ 0.0278 -0.0417  0.2222]]
rank(M)                                3
x = M\bb (square) → solve: (shape=(3,))
[-0.0556 -0.0556  0.6111]
x = Z\y (lstsq) → [intercept, slope]: (shape=(2,))
[2.9993 2.0008]
b/a → Solve A.T x.T = b.T, X shape     (2, 6)


<>:2: SyntaxWarning: invalid escape sequence '\ '
<>:2: SyntaxWarning: invalid escape sequence '\ '
/var/folders/v3/bk349gcs0fv4vrdxzx45yjh80000gn/T/ipykernel_49812/4154964099.py:2: SyntaxWarning: invalid escape sequence '\ '
  hdr("inv / pinv / rank / solves (\ and /)")


In [18]:
# ---- Factorizations & solvers -----------------------------------------------
hdr("Factorizations & solvers")
U, s, Vh = la.svd(M)
kv("svd: singular values", s)
print("chol(SPD):")
SPD3 = M.T @ M + 1e-6 * np.eye(3)
print(la.cholesky(SPD3))
w, V = la.eig(M)
kv("eig(M) eigenvalues", w)
Bspd = M.T @ M + 1e-3 * np.eye(3)
wg, Vg = la.eig(M, Bspd)
kv("eig(M,B) generalized eigenvalues (first 3)", wg[:3])
S = (M + M.T) / 2.0
wS, VS = la.eig(S)
kv("eig(S) dense eigenvalues", wS)
N_eigs = 50
L = diags(
    [2 * np.ones(N_eigs), -1 * np.ones(N_eigs - 1), -1 * np.ones(N_eigs - 1)],
    [0, -1, 1],
    format="csc",
)
w_eigs, V_eigs = eigs(L, k=3, which="LM")
kv("eigs(L,k=3) eigenvalues (largest magnitude)", np.real_if_close(w_eigs))
Q, R = la.qr(M)
kv("qr(M): diag(R)", np.diag(R))
from scipy.linalg import lu

P, Lfac, Ufac = lu(M)
kv("lu(M) shapes (P,L,U)", f"{P.shape}, {Lfac.shape}, {Ufac.shape}")
kv("||PLU - M||_F", la.norm(P @ Lfac @ Ufac - M, "fro"))
rhs = np.ones(3)
x_cg, info = cg(SPD3, rhs, maxiter=1000)
kv("cg SPD info (0=converged) & residual", f"{info}, {la.norm(SPD3 @ x_cg - rhs)}")




Factorizations & solvers
svd: singular values                   [8.8896 3.3201 2.4395]
chol(SPD):
[[3.6056 4.1603 3.8829]
 [0.     4.5489 3.2637]
 [0.     0.     4.3899]]
eig(M) eigenvalues                     [8.+0.j 3.+0.j 3.-0.j]
eig(M,B) generalized eigenvalues (first 3) [0.125 +0.j     0.3333+0.0013j 0.3333-0.0013j]
eig(S) dense eigenvalues               [8.6225+0.j 2.3775+0.j 3.    +0.j]
eigs(L,k=3) eigenvalues (largest magnitude) [3.9962 3.9848 3.9659]
qr(M): diag(R)                         [-3.6056 -4.5489  4.3899]
lu(M) shapes (P,L,U)                   (3, 3), (3, 3), (3, 3)
||PLU - M||_F                          0.0
cg SPD info (0=converged) & residual   0, 1.256515477855323e-14


In [19]:
# ---- FFT / iFFT --------------------------------------------------------------
hdr("FFT / iFFT")
sig = np.sin(np.linspace(0, 4 * np.pi, 32))
F = np.fft.fft(sig)
sig_rec = np.fft.ifft(F)
kv("len(FFT) & max reconstruction error", f"{len(F)}, {np.max(np.abs(sig - sig_rec))}")




FFT / iFFT
len(FFT) & max reconstruction error    32, 2.237726045655905e-16


In [20]:
# ---- Sorting -----------------------------------------------------------------
hdr("Sorting")
A_cols_sorted = np.sort(A[:5, :5], axis=0)
A_rows_sorted = np.sort(A[:5, :5], axis=1)
arr("sort(a) by column → np.sort(A, axis=0)", A_cols_sorted)
arr("sort(a,2) by row → np.sort(A, axis=1)", A_rows_sorted)
AR = np.array([[3, 9], [1, 5], [2, 7]])
I = np.argsort(AR[:, 0])
B_sorted = AR[I, :]
arr("[b,I]=sortrows(a,1) → I", I)
arr("[b,I]=sortrows(a,1) → b", B_sorted)




Sorting
sort(a) by column → np.sort(A, axis=0): (shape=(5, 5))
[[ 1.  2.  3.  4.  5.]
 [13. 14. 15. 16. 17.]
 [25. 26. 27. 28. 29.]
 [37. 38. 39. 40. 41.]
 [49. 50. 51. 52. 53.]]
sort(a,2) by row → np.sort(A, axis=1): (shape=(5, 5))
[[ 1.  2.  3.  4.  5.]
 [13. 14. 15. 16. 17.]
 [25. 26. 27. 28. 29.]
 [37. 38. 39. 40. 41.]
 [49. 50. 51. 52. 53.]]
[b,I]=sortrows(a,1) → I: (shape=(3,))
[1 2 0]
[b,I]=sortrows(a,1) → b: (shape=(3, 2))
[[1 5]
 [2 7]
 [3 9]]


In [21]:
# ---- Linear regression (x = Z\y) --------------------------------------------
hdr("Linear regression via lstsq")
Z = np.column_stack([np.ones(20), np.linspace(0, 1, 20)])
y = 4 - 2 * np.linspace(0, 1, 20) + 0.05 * np.random.randn(20)
coef2, *_ = la.lstsq(Z, y)
arr("lstsq coef ~ [intercept, slope]", coef2)




Linear regression via lstsq
lstsq coef ~ [intercept, slope]: (shape=(2,))
[ 4.0263 -2.0824]


In [22]:
# ---- Decimate / Unique / Squeeze --------------------------------------------
hdr("Decimate / Unique / Squeeze")
y_dec = signal.decimate(sig, 4, ftype="iir")
kv("decimate length: before → after", f"{len(sig)} → {len(y_dec)}")
arr("unique(a)", np.unique(np.array([[1, 1, 2], [2, 3, 3]])))
kv("squeeze(a) shape", np.zeros((3, 1, 4)).squeeze().shape)

print("\nAll MATLAB↔NumPy table items have been exercised. Done.\n")



Decimate / Unique / Squeeze
decimate length: before → after        32 → 8
unique(a): (shape=(3,))
[1 2 3]
squeeze(a) shape                       (3, 4)

All MATLAB↔NumPy table items have been exercised. Done.

